In [ ]:
import os,sys,warnings
if not sys.warnoptions:
    warnings.simplefilter('ignore')

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from plotly import tools
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.base import BaseEstimator, RegressorMixin,ClassifierMixin
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV
from sklearn.pipeline import Pipeline

#https://plotly.com/python/facet-plots/
#https://plotly.com/python/facet-plots/#wrapping-column-facets
 #   https://jakevdp.github.io/PythonDataScienceHandbook/03.11-working-with-time-series.html

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# https://www.dataquest.io/blog/tutorial-time-series-analysis-with-pandas/

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

def impute_knn(df):
    
    ''' inputs: pandas df containing feature matrix '''
    ''' outputs: dataframe with NaN imputed '''
    # imputation with KNN unsupervised method

    # separate dataframe into numerical/categorical
    ldf = df.select_dtypes(include=[np.number])           # select numerical columns in df
    ldf_putaside = df.select_dtypes(exclude=[np.number])  # select categorical columns in df
    # define columns w/ and w/o missing data
    cols_nan = ldf.columns[ldf.isna().any()].tolist()         # columns w/ nan 
    cols_no_nan = ldf.columns.difference(cols_nan).values     # columns w/o nan 

    for col in cols_nan:                
        imp_test = ldf[ldf[col].isna()]   # indicies which have missing data will become our test set
        imp_train = ldf.dropna()          # all indicies which which have no missing data 
        model = KNeighborsRegressor(n_neighbors=5)  # KNR Unsupervised Approach
        knr = model.fit(imp_train[cols_no_nan], imp_train[col])
        ldf.loc[df[col].isna(), col] = knr.predict(imp_test[cols_no_nan])
    
    return pd.concat([ldf,ldf_putaside],axis=1)

def bar_plot(x, y,palette_len,title='Missing Values (%)', xlim = None, ylim = None, 
             xticklabels = None, yticklabels = None,xlabel = None, ylabel = None, 
             figsize = (10,4),axis_grid = 'y'):
        
    cmap = sns.color_palette("plasma")
    fig, ax = plt.subplots(figsize = figsize)
    plt.title(title)

    for i in ['top', 'right', 'bottom', 'left']:
        ax.spines[i].set_color('black')
    
    ax.spines['top'].set_visible(True);ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False);ax.spines['left'].set_visible(False)

    sns.barplot(x = x, y = y, edgecolor = 'black', ax = ax,
                palette = cmap)
    ax.set_xlim(xlim);ax.set_ylim(ylim)    
    ax.set_xticklabels(xticklabels);ax.set_yticklabels(yticklabels)
    plt.xlabel(xlabel);plt.ylabel(ylabel)
    ax.grid(axis = axis_grid,ls='--',alpha = 0.9)
    plt.show()

# Plot Correlation Matrix
def corrMat(df,id=False,size=None):
    if(size is not None):
        figsize = (9,7)
    else:
        figsize = size
    corr_mat = df.corr().round(2)
    f, ax = plt.subplots(figsize=size)
    mask = np.triu(np.ones_like(corr_mat, dtype=np.bool))
    mask = mask[1:,:-1]
    corr = corr_mat.iloc[1:,:-1].copy()
    sns.heatmap(corr,mask=mask,vmin=-0.3,vmax=0.3,center=0, 
                cmap='Blues',square=False,lw=2,annot=True,cbar=False)
    ax.set_ylim(len(corr_mat)-0.5,-0.5)
    
# Plot Correlation to Target Variable only
def corrMat2(df,target='demand',figsize=(9,0.5),ret_id=False):
    
    corr_mat = df.corr().round(2);shape = corr_mat.shape[0]
    corr_mat = corr_mat.transpose()
    corr = corr_mat.loc[:, df.columns == target].transpose().copy()
    
    if(ret_id is False):
        f, ax = plt.subplots(figsize=figsize)
        sns.heatmap(corr,vmin=-0.3,vmax=0.3,center=0, 
                     cmap='Blues',square=False,lw=2,annot=True,cbar=False)
        plt.title(f'Feature Correlation to {target}')
    
    if(ret_id):
        return corr
    
''' Draw a Bivariate Seaborn Pairgrid /w KDE density w/ '''
def snsPairGrid(df,title):

    ''' Plots a Seaborn Pairgrid w/ KDE & scatter plot of df features'''
    sns.set(style='whitegrid')
    g = sns.PairGrid(df,diag_sharey=False)
    g.fig.set_size_inches(14,14)
    g.map_diag(sns.kdeplot, lw=2)
    g.map_lower(sns.scatterplot,s=15,edgecolor="k",linewidth=1.0,alpha=0.4)
    g.map_lower(sns.kdeplot,cmap='plasma',n_levels=3)
    g.set(xlim=(-0.2,0.2),ylim=(-0.2,0.2))
    g.fig.suptitle(title, y=1.02)
    plt.tight_layout()
    
def plot_na(ldf):
    naval = (ldf.isnull().sum()/len(ldf)*100).sort_values(ascending = False)
    bar_plot(x = naval,y = naval.index,
             palette_len = naval.index,xlim = (0,1),xticklabels = range(0,10,1),
             yticklabels = naval.index,figsize = (10,5), axis_grid = 'x')
        
# option to outline some regions in a plotly figure
def outline_plot(ldf=None,fig=None,var=None,level=0,mode='above',layer='below',verbose=False):

    assert(ldf is not None)
    assert(fig is not None)
    assert(var is not None)

    fillcolour = 'rgba(100,100,100,0.2)'
    if mode == 'above':
        m = ldf[var].gt(level)
    if mode == 'below':
        m = ldf[var].lt(level)

    ldf['Date'] = ldf.index
    df1 = ldf[m].groupby((~m).cumsum())['Date'].agg(['first','last'])

    for index, row in df1.iterrows():
        if(verbose):
            print(row['first'], row['last'])
        fig.add_shape(type="rect",xref="x", yref="paper",x0=row['first'],y0=0,
                        x1=row['last'],y1=1,line=dict(color="rgba(0,0,0,0)",width=3,),
                        fillcolor=fillcolour,layer=layer) 
    return(fig)

# One plot type plotly from dataframe
def plot_line(ldf,lst,title='',sec_id=None,height=350,trace_only=False,
              outline=False,add_bar=False,zoom=False,r_zoom=['2019','2020']):
    
    # sec_id - list of [False,False,True] values of when to activate 
    #           supblots; same length as lst
    
    # Always make new plot
    if(trace_only is False):
        if(sec_id is not None):
            fig = make_subplots(specs=[[{"secondary_y": True}]])
        else:
            fig = go.Figure()
        
    if(len(lst) is not 1):
        ii=-1
        for i in lst:
            ii+=1
            if(sec_id is not None):
                if(add_bar):
                    fig.add_bar(x=ldf.index,y=ldf[lst[ii]],name=lst[ii])
                else:
                    fig.add_trace(go.Scatter(x=ldf.index, y=ldf[lst[ii]],mode='lines',
                                         name=lst[ii],line=dict(width=2.0)),secondary_y=sec_id[ii])
            else:
                if(add_bar):
                    fig.add_bar(x=ldf.index, y=ldf[lst[ii]],name=lst[ii])
                else:
                    fig.add_trace(go.Scatter(x=ldf.index, y=ldf[lst[ii]],mode='lines',
                                             name=lst[ii],line=dict(width=2.0)))
    else:
        if(add_bar):
            fig.add_bar(x=ldf.index,y=ldf[lst[0]],name=lst[0])
        else:
            fig.add_trace(go.Scatter(x=ldf.index, y=ldf[lst[0]],mode='lines',
                                     name=lst[0],line=dict(width=2.0)))
            
    # Outline is an indicator of non final plot
    fig.update_layout(height=height,template='plotly_white',title=title,
                      margin=dict(l=0,r=0,t=30,b=0))
    if(zoom):
        fig.update_xaxes(type="date", range=[r_zoom[0],r_zoom[1]])
            
    if(outline is False):
        fig.show()
    else:
        return fig

# <span style='color:#5D2ECC'>Introduction</span>

- According to the **Clean Energy Australia Report 2020**, Victoria had a renewable energy penetration of 23.9% in 2019
which indicates that the majority comes from **fossil fuels** such as its coal-fired power stations [[1]](https://assets.cleanenergycouncil.org.au/documents/resources/reports/clean-energy-australia/clean-energy-australia-report-2020.pdf)
- Victoria’s 50 per cent renewable energy target by 2030 became law in October 2019, which indicates further transition into renewable energy will continue to occur in the next decade.




# <span style='color:#5D2ECC'>Feature Information</span>
A description of all the relevant features in the dataset

<b>date</b> : The date of the recording <br>
<b>school_day</b> : If students were at school on that day <br>
<b>holiday</b> : If the day was a state or national holiday <br>
    
## <span style='color:#5D6D7E'>Electricity Demand & Breakdown </span>

<b>demand</b> : Total daily electricity demand in MWh <br>
    
<b>RRP</b> : Recommended retail price in AUD/MWh <br>
<b>demand_pos_RRP</b> : A total daily demand at positive RRP in MWh <br>
<b>demand_neg_RRP</b> : Total daily demand at negative RRP in MWh <br>
    
<b>RRP_positive</b> : Averaged positive RRP, weighted by the corresponding intraday demand in AUD/MWh <br>
<b>RRP_negative</b> : Average negative RRP, weighted by the corresponding intraday demand in AUD/MWh <br>
<b>frac_at_neg_RRP</b> : A fraction of the day when the demand was traded at negative RRP <br>
    
## <span style='color:#5D6D7E'>Weather Related Features</span>
<b>min_temperature</b> : Minimum temperature during the day in Celsius <br>
<b>max_temperature</b> : Maximum temperature during the day in Celsius <br>
<b>solar_exposure</b> : Total daily sunlight energy in MJ/m^2 <br>
<b>rainfall</b> : Daily rainfall in mm <br>

September,October,November - Spring <br>
December,January,February - Summer <br>
March,April,May - Autumn <br>
June,July,August - Winter



# <span style='color:#5D2ECC'>Dataset Tasks</span>
*The dataset comes with two tasks:*

- **Electricity demand forecasting** is vital for our daily lives as it allows energy generation to match our needs without too much waste. How accurately is it possible to **forecast the demand for a day or a week ahead**?

- **Negative electricity** prices are damaging for electricity companies. Is it possible to predict them or determine the main factors responsible for **prolonged daily periods** of negative prices?

# <span style='color:#5D2ECC'>The Dataset</span>

Let's make some preliminary insights into the dataset:

- The dataset comes with __14 features__, one of which is date time.
- The dataset has a few missing features in columns **solar_exposure & rainfall** 
- Dataset time period: **1st JAN 2015** to **6th OCT 2020**
- Several features have very similar names (eg.`demand`, `demand_pos_RRP`, `demand_neg_RRP`); could have too high correlation to one another, which would make the model process pointless.
- Our features have a wide range of magnitudes, it will be difficult to compare them to one another without `scaling`
- Two features `school_day` & `holiday` use `Y/N`, which we'll replace with `1/0`

In [ ]:
path = '/kaggle/input/electricity-demand-in-victoria-australia/complete_dataset.csv'
df = pd.read_csv(path)
df.index = df['date']; del df['date']

# Let's use the 0/1 system instead of N/Y
df.replace({'school_day':{'N':0, 'Y':1}},inplace=True)
df.replace({'holiday':{'N':0, 'Y':1}},inplace=True)
df.index = pd.to_datetime(df.index)

# It's easier to work in GWh
pd.set_option("precision",5)
lst = ['demand','RRP','demand_pos_RRP','demand_neg_RRP','RRP_positive','RRP_negative']
df[lst].apply(lambda x: x/1000)

## <span style='color:#5D6D7E'>Missing Data</span>
- We only have missing data for <code>__rainfall__</code> & <code>__solar exposure__</code>, the rest of the data is available to us. 
- Missing data with kNN unsupervised learning methodology.

In [ ]:
sns.set(style='whitegrid',font_scale=1)
plot_na(df)

In [ ]:
# Let's show the missing data
pd.set_option("precision",3)
df[df.isna().any(axis=1)]

In [ ]:
# Finally Impute the missing data with knn
df_impute = impute_knn(df)
del df; df = df_impute
df.head()

# <span style='color:#5D2ECC'>Electricity demand forecasting</span>

Let's try the __first task__ in this dataset.

> **Electricity demand forecasting** is vital for our daily lives as it allows energy generation to match our needs without too much waste. How accurately is it possible to **forecast the demand for a day or a week ahead**?


In [ ]:
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.figure_factory as ff

def plotlyoff_corr(corr,size=None):
    
    xcols = corr.columns.tolist();ycols = xcols
    if(size is None):
        width = 700; height = 500
    else:
        width = size[0]; height = size[1]
    
    layout = dict(
        width = width,height = height,
        yaxis= dict(tickangle=-30,side = 'left'),
        xaxis= dict(tickangle=-30,side = 'top'))
    fig = ff.create_annotated_heatmap(
        z=corr.values,x= xcols,y= ycols,
        colorscale='viridis',showscale=False)
    fig['layout'].update(layout)
    fig.update_layout(margin={"r":0,"t":60,"l":0,"b":0})
    
    return py.iplot(fig)

## 2. <span style='color:#82409C'>Exploratory Data Analysis</span>
### <span style='color:#5D6D7E'>EDA: Pearson Correlation</span>
- <b>Demand</b> & <b>Demand_pos_RRP</b> are quite strongly correlated to one another (0.97) (let's plot them below)
- Most features a correlation of 0-0.26 to the target variable `demand`
- The higher the __demand__

In [ ]:
plotlyoff_corr(df_impute.corr().round(2))

We can see that both features are quite similar, so it makes little sense to keep it, let's remove it.

In [ ]:
plot_line(df['2019':'2021'],['demand','demand_pos_RRP'],height=250)

In [ ]:
df0 = df_impute.drop('demand_pos_RRP',axis=1)
df0.columns

### <span style='color:#5D6D7E'> RRP Relation to demand (Time Series) </span>
- Let's take a look at two resampled data relations, __5 days__ & __15 days__.
- We can note some correlation of RRP to demand, __mainy for spike demand peaks__, however when we look at the overall tren, there seems to be specific period during which, the <code>demand</code> can be roughly split into three quarters, __2015-2017__, during which it was lower, a period when it was higher, __2017-2020__, and again lower in 2020 onwards, however the demand doesn't quite change in the same way.
- Demand tends to be reducing, especially notable when we compare cyclic high demand peeks in __June-July__.

In [ ]:
df_0 = df0.copy()
conditions = [(df_0.index < '2016-12-31'),
             (df_0.index > '2017-01-01') & (df_0.index < '2019-12-01'),
             (df_0.index > '2019-12-2')]
values = ['0','1','2']
df_0['group'] = np.select(conditions, values)

In [ ]:
df0_trend = df0.resample('5D').mean()
plot_line(df0_trend,['demand','RRP'],sec_id=[False,True])

In [ ]:
df0_trend = df0.resample('6M').mean()
plot_line(df0_trend,['demand','RRP'],sec_id=[False,True])

### <span style='color:#5D6D7E'> RRP Relation to demand (Bivariate)</span>
Bivariate Relation of __RRP_positive__ & __RRP_negative__ to __demand__ (all data)

In [ ]:
titles = ['RRP_positive','RRP_negative']
fig = make_subplots(rows=1, cols=2,shared_yaxes=False,subplot_titles=None)

fig.add_trace(go.Scatter(y=df_0['RRP_positive'],x=df_0['demand'],color=df_0['group']),row=1,col=1)
fig.add_trace(go.Scatter(y=df_0['RRP_negative'],x=df_0['demand']),row=1, col=2)

fig.update_traces(mode='markers', marker_line_width=1, marker_size=5)
fig.update_layout(template='plotly_white',height=500,showlegend=True)
fig.update_layout(margin={"r":0,"t":60,"l":0,"b":0})
fig.update_xaxes(title_text='demand')
fig.update_yaxes(range=[0,400], row=1, col=1)
fig.update_yaxes(range=[-100,0], row=1, col=2)
fig.show()

## <span style='color:#F1C40F'> Preliminary Models </span>

> **Electricity demand forecasting** is vital for our daily lives as it allows energy generation to match our needs without too much waste. How accurately is it possible to **forecast the demand for a day or a week ahead**

### <span style='color:#5D6D7E'> Cross Validation on Training Data </span>
Firstly evaluating the cross_val_score on the training set

In [ ]:
# Split for TimeSeries
def TimeSeries_Split(ldf,feature=None,split_id=[None,None],cut_id=None,plot_id=False,show_eval=False):
    
    # Reduce the number of used data
    if(cut_id is not None):
        print('Dataset Maximum Index Number Is Reduced')
        ldf = ldf.iloc[-cut_id:]
        t1 = ldf.index.max();t0 = ldf.index.min()
        print(f'Dataset Min.Index: {t0} | Max.Index: {t1}')
        
    if(split_id[0] is not None):
        # General Percentage Split (Non Shuffle requied for Time Series)
        train_df,eval_df = train_test_split(ldf,test_size=split_id[0],shuffle=False)
    elif(split_id[1] is not None):
        # specific time split 
        train_df = df.loc[:split_id[1]]; eval_df = df.loc[split_id[1]:] 
    else:
        print('Choose One Splitting Method Only')
        
    if(plot_id):
         
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=train_df.index, y=train_df[feature],
                                 mode='lines',name='Training Data', line={'width': 2}))
        fig.update_layout(template='plotly_white',title='Training Signal Visualisation',
                          height=300,margin=dict(l=50,r=80,t=50,b=40))
        if(show_eval):
            fig.add_trace(go.Scatter(x=eval_df.index, y=eval_df[feature],
                                    mode='lines',name='Test Data',line={'width': 2}))
            fig.update_layout(title='Training/Test Signal Visualisation')
        fig.show()
        
    return train_df,eval_df # return 

In [ ]:
tr_data1, te_data1 = TimeSeries_Split(df_impute2,split_id=[0.1,None],plot_id=True,feature='demand')

In [ ]:
''' Features '''
tr_data1.columns

In [ ]:
# Model Evaluation w/ Cross Validation
def eval1(ldf,feature='demand',eval_id='cv'):
    
    # Input: Feature & Target DataFrame
    if(eval_id is 'trte'):
        data1,data2 = TimeSeries_Split(ldf,split_id=[0.1,None])

    # Split feature/target variable
    y = ldf[feature].copy()
    X = ldf.copy()
    del X[feature]     # remove target variable
    
    # Pick Model 
    model = RandomForestRegressor(n_estimators=100,random_state=10)
    if(eval_id is 'cv'):
        cv_score = -cross_val_score(model,X,y,cv=5,scoring='neg_mean_squared_error')
        print(f'cross_val_score: {cv_score.round(2)}')
        print(f'cross_val_score mean: {cv_score.mean().round(2)}')
    if(eval_id is 'trte'):
        y1 = data1[feature].copy(); X1 = data1.copy(); del X1[feature]
        y2 = data2[feature].copy(); X2 = data2.copy(); del X2[feature]
        model.fit(X1,y1)
        y_train = model.predict(X1); y_eval = model.predict(X2)
        data1['train_model'] = y_train; data2['eval_model'] = y_eval
        plot_line(data1,[feature,'train_model'])
        plot_line(data2,[feature,'eval_model'])

In [ ]:
eval1(tr_data1,eval_id='cv')

In [ ]:
eval1(df_impute2,eval_id='trte')

In [ ]:
# from scipy.fftpack import fft
# import plotly.graph_objects as go
# from plotly.subplots import make_subplots
# import scipy

# def plot_fft(ldf,name):
#     signal_noise = ldf[name].values
#     N = len(ldf[name])
#     T = 1/N

#     x = np.linspace(0.0, 1.0/(2.0*T), int(N/2))
#     yr = fft(signal_noise) # "raw" FFT with both + and - frequencies
#     y = 2/N * np.abs(yr[0:np.int(N/2)]) # positive freqs only
#     return x[2:],y[2:]

# # Create figure with secondary y-axis
# fig = make_subplots(specs=[[{"secondary_y": True}]])

# x1, y1 = plot_fft(RRP_feat_df,'solar_exposure')
# x1, y2 = plot_fft(RRP_feat_df,'demand_neg_RRP')

# # Add traces
# fig.add_trace(go.Scatter(x=x1, y=y1, name="yaxis data"),secondary_y=False)
# fig.add_trace(go.Scatter(x=x1, y=y2, name="yaxis data"),secondary_y=True)
# fig.update_layout(height=300, width=1000,margin=dict(l=40, r=100, t=40, b=30),title='Weather Features : Frequency Domain') 
# fig.show()